# Armadillo ASE analysis (200219)
Simple model (common p for each quad for each SNP) and pair comparison

In [54]:
require(viridis)
require(ggplot2)
require(dplyr)
require(parallel)

set.seed(42)

In [55]:
load("data/exprs_all.Rdata")
load("data/metadata.Rdata")
#source("data/useful.r")
load("data/ase_ratios.test_train.Rdata")
#load("data/gene_annotations_v0.95.Rdata")
#load("data/armadillo.helper.Rdata")

In [56]:
print(pData$Sex)
sexlabels
timelabels

 [1] M M M M F F F F F F F F F F F F M M M M M M M M F F F F F F F F F F F F M M
[39] M M M M M M F F F F F F F F F F F F M M M M
Levels: F M


[1] 2 2 2 1 1 1 1 1 1 1 1 1 2 2 2

[1] 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3

In [57]:
# Variables 
n_quads = 5
n_times = 3 
n_qt = n_quads * n_times
n_q = 4 
n_samp = n_quads * n_q
r_samp = 1:n_samp 

# Labels 
quad = as.numeric(pData$Quad)
sex = as.numeric(pData$Sex)
lane = as.numeric(pData$Batches)
tlab = c("t1", "t2", "t3")
quads = unique(substr(pData$ID, 1,4))

labels = paste(sapply(1:n_quads, function(i) rep(quads[i],n_times)), tlab  ) 
sexlabels = unlist(lapply(1:n_quads, function(i) rep(unique(cbind(pData$Quad, pData$Sex) )[i,2],n_times)) ) 
timelabels = rep(1:n_times, n_quads)
quadlabels = unlist(lapply(1:n_quads, function(i) rep(unique(cbind(pData$Quad, pData$Sex) )[i,1],n_times))   ) 


In [58]:
exprs.all.filt = lapply( 1:5, function(i) exprs.all[[i]][!is.na(exprs.all[[i]][,1]),]) 
ratios = list() 
for(j in 1:5){ 
  X.temp = (exprs.all.filt[[j]][,7:18]) 
  nj = dim(X.temp)[1]/2
  ratios[[j]] = cbind((X.temp[(1:nj),1:4] / (X.temp[(1:nj)+nj,1:4] +X.temp[(1:nj),1:4] ) ),
                      (X.temp[(1:nj),5:8] / (X.temp[(1:nj)+nj,5:8] +X.temp[(1:nj),5:8] ) ),
                      (X.temp[(1:nj),9:12] / (X.temp[(1:nj)+nj,9:12] +X.temp[(1:nj),9:12] ) ))
  
}
density.plot = lapply(1:5, function(i) density( ratios[[i]][!is.na(ratios[[i]])] )  )

In [59]:
ase_cov = list()
for (i in 1:5) {
    X.temp = exprs.all.filt[[i]][,7:18]
    nj = dim(X.temp)[1]/2
    allele_a = X.temp[1:nj,]
    allele_b = X.temp[1:nj+nj,]
    ase_cov[[i]] = lapply(1:12, function(j) {
        df = data.frame(major=allele_a[,j], minor=allele_b[,j])
        df = cbind(df, cov=rowSums(df))
        df = cbind(df, ratio=allele_a[,j]/df$cov)
        return(df)
    })
}

In [60]:
summarize.one.pair.data.simple <- function(data) {
    tdf <- data %>% group_by(gene) %>% summarise(
       mean_cov = mean(cov),
       cor = min(cor),
       scor = min(scor),
       prob = sum(prob, na.rm=TRUE),
       pvalue  = sum(pvalue, na.rm=TRUE),
       prob_fixed = sum(prob_fixed, na.rm=TRUE),
        pvalue_fixed = sum(pvalue_fixed, na.rm=TRUE),
        p = min(p),
        p_fixed = min(p_fixed)
    )
    tdf <- cbind(tdf, exp_pvalue = exp(tdf$pvalue))
    tdf <- cbind(tdf, exp_pvalue_fixed = exp(tdf$pvalue_fixed))
    tdf <- cbind(tdf, exp_prob=exp(tdf$prob))
    tdf <- cbind(tdf, exp_prob_fixed=exp(tdf$prob_fixed))
    return(tdf)
}
summarize.one.data.simple <- function(data) {
    tdf <- data %>% group_by(gene) %>% summarise(
       mean_cov = mean(cov),
       prob = sum(prob, na.rm=TRUE),
       pvalue  = sum(pvalue, na.rm=TRUE),
       prob_fixed = sum(prob_fixed, na.rm=TRUE),
       pvalue_fixed = sum(pvalue_fixed, na.rm=TRUE),
       p = min(p),
       p_fixed = min(p_fixed)      
    )
    return(tdf)
}

In [61]:
estimate.p.for.quad <- function(ase_cov, ident, time, g) {
    n = sum(unlist(lapply(1:4, function(x){return(ase_cov[[ident]][[time*4+x]][g, 3])})), na.rm=T)
    ny = sum(unlist(lapply(1:4, function(x){return(ase_cov[[ident]][[time*4+x]][g, 1])})), na.rm=T)
    return(ny/n)
}
estimate.p.for.quad.fixed <- function(ase_cov, ident, time, g) {
    return(mean(estimate.p.for.each(ase_cov, ident, time, g), na.rm=T))
}
estimate.p.for.each <- function(ase_cov, ident, time, g) {
    return(unlist(lapply(1:4, function(x){return(ase_cov[[ident]][[time*4+x]][g,4])})))
}

In [62]:
compute.prob <- function(major, cov, p) {
    return(unlist(sapply(1:4, function(x) {return(dbinom(major[x], cov[x], p, log=TRUE))})))
}
compute.pvalue <- function(major, cov, p) {
    return(unlist(sapply(1:4, function(x) {
                #if (cov[x] == 0) return(1);
                if (cov[x] == 0) return(0.5);
                if (major[x]/cov[x] < p) return(pbinom(major[x], cov[x], p, log=TRUE, lower.tail=TRUE))
                else return(pbinom(major[x], cov[x], p, log=TRUE, lower.tail=FALSE))
    })))
}


In [63]:
compute.simple.pvalue.comb <- function(ase_cov, ident, time, gene_set=NULL, verbose=FALSE) {
    if (is.null(gene_set))
        gene_set = 1:dim(ase_cov[[ident]][[1]][1])[1]
    df <- do.call(rbind, lapply(gene_set, function(g) {
        x_cord <- unlist(lapply(1:4, function(x){return(ase_cov[[ident]][[x]][g,4])}))
        x_cov <- unlist(lapply(1:4, function(x){return(ase_cov[[ident]][[x]][g,3])}))
        y_cord <- unlist(lapply(1:4, function(x){return(ase_cov[[ident]][[time*4+x]][g,4])}))
        y_major <- unlist(lapply(1:4, function(x){return(ase_cov[[ident]][[time*4+x]][g,1])}))
        y_cov <- unlist(lapply(1:4, function(x){return(ase_cov[[ident]][[time*4+x]][g,3])}))
        if (sum(is.nan(x_cord+y_cord)) >= 2)
            return(NULL)
        p <- estimate.p.for.quad(ase_cov, ident, 0, g)
        if (is.na(p))
            return(NULL)
        prob = compute.prob(y_major, y_cov, p)
        pvalue = compute.pvalue(y_major, y_cov, p)
        p_fixed <- estimate.p.for.quad.fixed(ase_cov, ident, 0, g)
        if (!is.na(p_fixed)) {
            prob_fixed = compute.prob(y_major, y_cov, p_fixed)
            pvalue_fixed = compute.pvalue(y_major, y_cov, p_fixed)
        }
#         y_cord <- y_cord[order(x_cord)]
#         x_cov <- x_cov[order(x_cord)]
#         y_cov <- y_cov[order(x_cord)]
#         x_cord <- x_cord[order(x_cord)]
        suppressWarnings({
            temp = data.frame(x=x_cord, y=y_cord, gene=g, cor=rep(cor.test(x=x_cord, y=y_cord, method = 'pearson')$estimate, 4), 
                              scor=rep(cor.test(x=x_cord, y=y_cord, method = 'spearman')$estimate, 4), cov=(x_cov+y_cov)/2, 
                              prob=prob, pvalue=pvalue, prob_fixed=prob_fixed, pvalue_fixed=pvalue_fixed, p=rep(p, 4), p_fixed=rep(p_fixed, 4))
            temp <- temp[order(x_cord),]
        })
        return(temp)
    }))
    return(df)
}
compute.simple.pvalue.comb.parallel <- function(ident) {
    result <- list()
    for (j in 1:2) {
        gene_set = sample(1:dim(ase_cov[[ident]][[j]][1])[1], 2000)
        result[[j]] <- compute.simple.pvalue.comb(ase_cov, ident, j)
    }
    return(result)
}



In [64]:
sdf <- mclapply(1:5, compute.simple.pvalue.comb.parallel, mc.cores=5)
# sdf[[5]] <- compute.simple.pvalue.comb.parallel(5)
# sdf <- mclapply(1:1, compute.simple.pvalue.parallel, mc.cores=1)
print(names(sdf))
print(length(sdf))
print(length(sdf[[1]]))
print(head(sdf[[1]][[1]]))
print(head(sdf[[5]][[1]]))
saveRDS(sdf, 'simple_t0_result.rds')

NULL
[1] 5
[1] 2
           x         y gene         cor       scor  cov      prob    pvalue
3  0.2222222 0.7692308    1 -0.27642212  0.1054093 11.0 -3.851235 -5.139387
2  0.4444444 0.3181818    1 -0.27642212  0.1054093 15.5 -2.717362 -2.138473
4  0.5333333 0.4000000    1 -0.27642212  0.1054093 17.5 -1.896625 -1.031230
1  0.5833333 0.7692308    1 -0.27642212  0.1054093 12.5 -3.851235 -5.139387
11 0.3181818 0.7857143    2 -0.05075261 -0.4000000 18.0 -5.068150 -6.619846
41 0.3809524 0.4666667    2 -0.05075261 -0.4000000 25.5 -2.013288 -1.284049
   prob_fixed pvalue_fixed         p   p_fixed
3   -4.192978   -5.5766652 0.4666667 0.4458333
2   -2.462269   -1.8252542 0.4666667 0.4458333
4   -1.802159   -0.8460045 0.4666667 0.4458333
1   -4.192978   -5.5766652 0.4666667 0.4458333
11  -5.046982   -6.5939290 0.4302326 0.4312771
41  -2.008699   -1.2699573 0.4302326 0.4312771
             x           y gene       cor       scor   cov      prob     pvalue
2  0.000000000 0.000000000    1 -0.516761 

In [49]:
for (i in 1:5) {
    for (j in 1:2) {
        print(head(sdf[[i]][[j]]))
    }
}


           x         y gene         cor       scor  cov      prob       pvalue
3  0.2222222 0.7692308    1 -0.27642212  0.1054093 11.0 -3.851235 -0.005878527
2  0.4444444 0.3181818    1 -0.27642212  0.1054093 15.5 -2.717362 -2.138473287
4  0.5333333 0.4000000    1 -0.27642212  0.1054093 17.5 -1.896625 -1.031230128
1  0.5833333 0.7692308    1 -0.27642212  0.1054093 12.5 -3.851235 -0.005878527
11 0.3181818 0.7857143    2 -0.05075261 -0.4000000 18.0 -5.068150 -0.001334526
41 0.3809524 0.4666667    2 -0.05075261 -0.4000000 25.5 -2.013288 -0.324226849
   prob_fixed pvalue_fixed         p   p_fixed
3   -4.192978 -0.003792349 0.4666667 0.4458333
2   -2.462269 -1.825254156 0.4666667 0.4458333
4   -1.802159 -0.846004534 0.4666667 0.4458333
1   -4.192978 -0.003792349 0.4666667 0.4458333
11  -5.046982 -0.001369589 0.4302326 0.4312771
41  -2.008699 -0.329676432 0.4302326 0.4312771
           x         y gene        cor scor  cov      prob      pvalue
3  0.2222222 0.3333333    1  0.4660849  0.2 10.

In [99]:
print(head(exprs.all.filt[[1]]))
print(head(ase_cov[[1]][[1]]))

             ensemblID           type name     chrm    pos allele 15-501_1
175 ENSDNOG00000042290 protein_coding  SYK JH563979 766458      G        7
176 ENSDNOG00000042290 protein_coding  SYK JH563979 766677      G        7
177 ENSDNOG00000042290 protein_coding  SYK JH563979 766712      G        3
179 ENSDNOG00000042290 protein_coding  SYK JH563979 766874      G        6
181 ENSDNOG00000042290 protein_coding  SYK JH563979 767289      T        4
182 ENSDNOG00000042290 protein_coding  SYK JH563979 767371      T       13
    15-502_1 15-503_1 15-504_1 15-501_2 15-502_2 15-503_2 15-504_2 15-501_3
175        4        2        8       10        7       10        8        3
176       10       12        8       11        5       11       14        7
177        5       12       10       14        5       10       10        9
179        7        6       10       15       11        8       19        3
181       11        6        4        9       18        7       14        4
182        5       

In [65]:
extract.gene.info <- function(genes, ident) {
    return(exprs.all.filt[[ident]][genes,1:5])
}

In [66]:
plot_histogram <- function(data, x, header, quant_value=-1) {
    png(paste0(header, '.png'))
    g <- ggplot(data, aes_string(x=x))+geom_histogram()+theme_bw()
    if (quant_value > 0) {
        q <- quantile(data[,x], quant_value)
        print(q)
        g <- g + geom_vline(xintercept=q, color='red', size=2) +
        geom_text(label = paste(quant_value, ':', q), x=q, y=-50)
    }
    plot(g) 
    dev.off()
}

In [67]:
all_df <- NULL
sig_df <- NULL
data <- sdf
for (i in 1:5) {
    for (j in 1:length(data[[i]])) {
        tdf <- summarize.one.pair.data.simple(data[[i]][[j]])
        new_df <- data.frame(gene=tdf$gene, prob=tdf$prob, pvalue=tdf$pvalue, prob_fixed=tdf$prob_fixed, pvalue_fixed=tdf$pvalue_fixed, mean_cov=tdf$mean_cov)
        for (val in c('prob', 'pvalue', 'pvalue_fixed', 'prob_fixed')) {
            if (any(val == c('prob', 'prob_fixed')))
                new_df[new_df[,val] < -50, val] = -50
            else
                new_df[new_df[,val] < -30, val] = -30 
        }
        plot_histogram(new_df, 'prob', paste0('hist_prob_t0_null_', i, '_', j), quant_value=0.01)
        plot_histogram(new_df, 'pvalue', paste0('hist_pvalue_t0_null_', i, '_', j), quant_value=0.01)
        tdf <- subset(new_df, new_df$pvalue < quantile(new_df$pvalue, 0.01))
        tdf <- cbind(tdf, extract.gene.info(tdf$gene, i))
        tdf <- cbind(tdf, data.frame(ident=rep(i, dim(tdf)[1]), time=rep(j, dim(tdf)[1])))        
        if (is.null(all_df)) all_df <- tdf
        else all_df <- rbind(all_df, tdf)
        new_sdf <- subset(new_df, new_df$pvalue < quantile(new_df$pvalue, 0.01))
        new_sdf <- cbind(new_sdf, extract.gene.info(new_sdf$gene, i))
        new_sdf <- cbind(new_sdf, data.frame(ident=rep(i, dim(new_sdf)[1]), time=rep(j, dim(new_sdf)[1])))
        if (is.null(sig_df)) sig_df <- new_sdf
        else sig_df <- rbind(sig_df, new_sdf)
            
    }
}

       1% 
-21.74035 


`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.



      1% 
-22.0199 


`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.



       1% 
-19.39961 


...

`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.



       1% 
-22.15507 


`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.



       1% 
-25.55472 


`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.



In [68]:
print(head(sig_df))
saveRDS(all_df, 'simple_t0_all_summary.rds')
saveRDS(sig_df, 'simple_t0_sig_summary.rds')

    gene      prob    pvalue prob_fixed pvalue_fixed mean_cov
491  491   0.00000 -30.00000    0.00000    -30.00000   33.625
585  585 -25.93574 -22.88149  -22.40556    -19.35979   54.375
641  641 -23.63195 -23.45608  -24.26315    -24.15789   40.250
647  647 -50.00000 -30.00000  -50.00000    -30.00000  302.250
648  648 -50.00000 -30.00000  -50.00000    -30.00000  301.625
649  649 -50.00000 -30.00000  -50.00000    -30.00000  302.375
             ensemblID           type    name     chrm    pos ident time
491 ENSDNOG00000051200        lincRNA         JH564093  73771     1    1
585 ENSDNOG00000035200 protein_coding CLEC12A JH564093 408835     1    1
641 ENSDNOG00000035200 protein_coding CLEC12A JH564093 414188     1    1
647 ENSDNOG00000035200 protein_coding CLEC12A JH564093 416698     1    1
648 ENSDNOG00000035200 protein_coding CLEC12A JH564093 416699     1    1
649 ENSDNOG00000035200 protein_coding CLEC12A JH564093 416703     1    1
